This notebook is only for training the models with the best parametrs found in `3_model_training`. If you don't need the trained models, you can skip to `4_evaluation`.

In [10]:
import os

from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.metrics import f1_score, accuracy_score, recall_score, precision_score
import pandas as pd
import numpy as np
import xgboost as xgb
from sklearn.preprocessing import StandardScaler

os.sys.path.append(os.path.abspath('../src'))
from data import preprocessor

mapname = 'Town01'

In [106]:
def create_and_train_model(model, args):
    """
    Function to create and train a model with given parameters.

    Args:
        model: The model to be trained.
        args: Arguments for training the model.

    Returns:
        result: The result of the training process.
    """
    result = {}
    # Train model with current hyperparameters
    md = model(**args)
    md.fit(X_train, y_train)
    # Predict on the validation set
    y_pred = md.predict(X_val)
    # Log training results
    result["f1_score"] = f1_score(y_val, y_pred, average='weighted')
    result["accuracy"] = accuracy_score(y_val, y_pred)
    result["recall"] = recall_score(y_val, y_pred, average='weighted')
    result["precision"] = precision_score(y_val, y_pred, average='weighted')
    result["model"] = md

    return result

In [108]:
def add_to_dict(dict, key, model, results):
    """
    Function to add results to the dictionary.

    Args:
        dict: The dictionary to which results will be added.
        key: The key for the dictionary.
        model: The model used.
        results: The results of the training process.

    Returns:
        None
    """
    if key not in dict:
        dict[key] = {
            'model': model,
            'f1_scores': [],
            'accuracy': [],
            'recall': [],
            'precision': [],
        }
    
    dict[key]['f1_scores'].append(results['f1_score'])
    dict[key]['accuracy'].append(results['accuracy'])
    dict[key]['recall'].append(results['recall'])
    dict[key]['precision'].append(results['precision'])

## Preparing the Data

In [4]:
data_path = '../data'
uah_training = pd.read_csv(f'{data_path}/base/training_set_uah.csv')
uah_validation = pd.read_csv(f'{data_path}/base/validation_set_uah.csv')

carla_fixed = pd.read_csv(f'{data_path}/merged/{mapname}/carla/carla_fixed.csv').drop(columns=['origin'])
carla_llm = pd.read_csv(f'{data_path}/merged/{mapname}/carla/carla_llm.csv').drop(columns=['origin'])

sumo_fixed = pd.read_csv(f'{data_path}/merged/{mapname}/sumo/sumo_fixed.csv').drop(columns=['origin'])
sumo_llm = pd.read_csv(f'{data_path}/merged/{mapname}/sumo/sumo_llm.csv').drop(columns=['origin'])

carla_uah_fixed = pd.read_csv(f'{data_path}/merged/{mapname}/carla_uah/carla_uah_fixed.csv')
carla_uah_llm = pd.read_csv(f'{data_path}/merged/{mapname}/carla_uah/carla_uah_llm.csv')

sumo_uah_fixed = pd.read_csv(f'{data_path}/merged/{mapname}/sumo_uah/sumo_uah_fixed.csv')
sumo_uah_llm = pd.read_csv(f'{data_path}/merged/{mapname}/sumo_uah/sumo_uah_llm.csv')

In [5]:
sumo_columns_to_keep = sumo_fixed.columns.tolist()
carla_columns_to_keep = carla_fixed.columns.tolist()

In [6]:
window_size = 10
step_size = 5
one_hot_keys = {
    'normal': 0,
    'aggressive': 1
}

# Train on Real test on Synthetic

By training a model on real data and testing on synthetic data, we verify synthetic data’s fidelity with the real-world.

In [86]:
TRTS = {}
n_trials = 8

## SUMO

In [87]:
X_train, y_train = preprocessor.sliding_windows(uah_training[sumo_columns_to_keep], window_size=10, step_size=5)
y_train = preprocessor.one_hot_encode(y_train, one_hot_keys)

### Fixed

In [88]:
X_val, y_val = preprocessor.sliding_windows(sumo_fixed, window_size=10, step_size=5)
y_val = preprocessor.one_hot_encode(y_val, one_hot_keys)

In [ ]:
# Best model: XGBClassifier_{'learning_rate': 0.001, 'max_depth': 10, 'n_estimators': 1000}
for i in range(n_trials):
    results = create_and_train_model(xgb.XGBClassifier, {
        'learning_rate': 0.001,
        'max_depth': 10,
        'n_estimators': 1000,
    })
    add_to_dict(TRTS, 'sumo_fixed', 'XGBClassifier', results)

Params: {'learning_rate': 0.001, 'max_depth': 10, 'n_estimators': 1000}
Params: {'learning_rate': 0.001, 'max_depth': 10, 'n_estimators': 1000}
Params: {'learning_rate': 0.001, 'max_depth': 10, 'n_estimators': 1000}
Params: {'learning_rate': 0.001, 'max_depth': 10, 'n_estimators': 1000}
Params: {'learning_rate': 0.001, 'max_depth': 10, 'n_estimators': 1000}
Params: {'learning_rate': 0.001, 'max_depth': 10, 'n_estimators': 1000}
Params: {'learning_rate': 0.001, 'max_depth': 10, 'n_estimators': 1000}
Params: {'learning_rate': 0.001, 'max_depth': 10, 'n_estimators': 1000}


### LLM

In [92]:
X_val, y_val = preprocessor.sliding_windows(sumo_llm, window_size=10, step_size=5)
y_val = preprocessor.one_hot_encode(y_val, one_hot_keys)

In [ ]:
# Best model: RandomForestClassifier_{'max_depth': 10, 'n_estimators': 50}
for i in range(n_trials):
    results = create_and_train_model(RandomForestClassifier, {
        'max_depth': 10,
        'n_estimators': 50,
    })
    add_to_dict(TRTS, 'sumo_llm', 'RFClassifier', results)

Params: {'max_depth': 10, 'n_estimators': 50}
Params: {'max_depth': 10, 'n_estimators': 50}
Params: {'max_depth': 10, 'n_estimators': 50}
Params: {'max_depth': 10, 'n_estimators': 50}
Params: {'max_depth': 10, 'n_estimators': 50}
Params: {'max_depth': 10, 'n_estimators': 50}
Params: {'max_depth': 10, 'n_estimators': 50}
Params: {'max_depth': 10, 'n_estimators': 50}


## CARLA

In [94]:
X_train, y_train = preprocessor.sliding_windows(uah_training[carla_columns_to_keep], window_size=10, step_size=5)
y_train = preprocessor.one_hot_encode(y_train, one_hot_keys)

### Fixed

In [95]:
X_val, y_val = preprocessor.sliding_windows(carla_fixed, window_size=10, step_size=5)
y_val = preprocessor.one_hot_encode(y_val, one_hot_keys)

In [ ]:
# Best model: XGBClassifier_{'learning_rate': 0.2, 'max_depth': 5, 'n_estimators': 700}
for i in range(n_trials):
    results = create_and_train_model(xgb.XGBClassifier, {
        'learning_rate': 0.2,
        'max_depth': 5,
        'n_estimators': 700,
    })
    add_to_dict(TRTS, 'carla_fixed', 'XGBClassifier', results)

Params: {'learning_rate': 0.2, 'max_depth': 5, 'n_estimators': 700}
Params: {'learning_rate': 0.2, 'max_depth': 5, 'n_estimators': 700}
Params: {'learning_rate': 0.2, 'max_depth': 5, 'n_estimators': 700}
Params: {'learning_rate': 0.2, 'max_depth': 5, 'n_estimators': 700}
Params: {'learning_rate': 0.2, 'max_depth': 5, 'n_estimators': 700}
Params: {'learning_rate': 0.2, 'max_depth': 5, 'n_estimators': 700}
Params: {'learning_rate': 0.2, 'max_depth': 5, 'n_estimators': 700}
Params: {'learning_rate': 0.2, 'max_depth': 5, 'n_estimators': 700}


### LLM

In [97]:
X_val, y_val = preprocessor.sliding_windows(carla_llm, window_size=10, step_size=5)
y_val = preprocessor.one_hot_encode(y_val, one_hot_keys)

In [ ]:
# Best model: XGBClassifier_{'learning_rate': 0.3, 'max_depth': 5, 'n_estimators': 500}
for i in range(n_trials):
    results = create_and_train_model(xgb.XGBClassifier, {
        'learning_rate': 0.3,
        'max_depth': 5,
        'n_estimators': 500,
    })
    add_to_dict(TRTS, 'carla_llm', 'XGBClassifier', results)

Trial 1: F1 Score = 0.5194390536550584
Trial 2: F1 Score = 0.5194390536550584
Trial 3: F1 Score = 0.5194390536550584
Trial 4: F1 Score = 0.5194390536550584


KeyboardInterrupt: 

## Results table

In [101]:
for key, value in TRTS.items():
    print(f"{key}:")
    print(f"  Model: {value['model']}")
    print(f"  F1 Score: {np.mean(value['f1_scores']):.2f} ± {np.std(value['f1_scores']):.2f}")
    print(f"  Accuracy: {np.mean(value['accuracy']):.2f} ± {np.std(value['accuracy']):.2f}")
    print(f"  Recall: {np.mean(value['recall']):.2f} ± {np.std(value['recall']):.2f}")
    print(f"  Precision: {np.mean(value['precision']):.2f} ± {np.std(value['precision']):.2f}\n")

sumo_fixed:
  Model: XGBClassifier
  F1 Score: 0.47 ± 0.00
  Accuracy: 0.50 ± 0.00
  Recall: 0.50 ± 0.00
  Precision: 0.50 ± 0.00

sumo_llm:
  Model: RandomForestClassifier
  F1 Score: 0.48 ± 0.00
  Accuracy: 0.52 ± 0.00
  Recall: 0.52 ± 0.00
  Precision: 0.51 ± 0.00

carla_fixed:
  Model: XGBClassifier
  F1 Score: 0.53 ± 0.00
  Accuracy: 0.53 ± 0.00
  Recall: 0.53 ± 0.00
  Precision: 0.53 ± 0.00

carla_llm:
  Model: XGBClassifier
  F1 Score: 0.52 ± 0.00
  Accuracy: 0.52 ± 0.00
  Recall: 0.52 ± 0.00
  Precision: 0.53 ± 0.00



# Discriminative Score

We seek to understand how well a classifier can separate the real and the synthetic data to determine how indistiguishable they are.

If the classifiers do not get good scores, that means the data is hardly distinguishable.

In [152]:
X_uah_disc_sumo = uah_training[sumo_columns_to_keep].drop(columns=['label'])
X_uah_disc_carla = uah_training[carla_columns_to_keep].drop(columns=['label'])
y_uah_disc = np.zeros(len(X_uah_disc_carla)) # 0 for real data and 1 for synthetic data

## SUMO

In [156]:
X_sumo_disc_fixed, y_sumo_disc_fixed = sumo_fixed.drop(columns=['label']), np.ones(len(sumo_fixed))
X_sumo_disc_llm, y_sumo_disc_llm = sumo_llm.drop(columns=['label']), np.ones(len(sumo_llm))

### Fixed

In [206]:
X, y = pd.concat([X_sumo_disc_fixed, X_uah_disc_sumo], axis=0, ignore_index=True), np.concat([y_sumo_disc_fixed, y_uah_disc], axis=0)
X.reset_index(drop=True, inplace=True)
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42, shuffle=True)

In [207]:
rf = RandomForestClassifier()
rf.fit(X_train, y_train)
y_pred = rf.predict(X_val)
print(f"F1 Score: {f1_score(y_val, y_pred, average='weighted')}")
print(f"Accuracy: {accuracy_score(y_val, y_pred)}")

F1 Score: 1.0
Accuracy: 1.0


In [215]:
svc = SVC(kernel='rbf')
svc.fit(X_train, y_train)
y_pred = svc.predict(X_val)
print(f"F1 Score: {f1_score(y_val, y_pred, average='weighted')}")
print(f"Accuracy: {accuracy_score(y_val, y_pred)}")

F1 Score: 1.0
Accuracy: 1.0


### LLM

In [209]:
X, y = pd.concat([X_sumo_disc_llm, X_uah_disc_sumo], axis=0, ignore_index=True), np.concat([y_sumo_disc_llm, y_uah_disc], axis=0)
X.reset_index(drop=True, inplace=True)
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42, shuffle=True)

In [210]:
rf = RandomForestClassifier()
rf.fit(X_train, y_train)
y_pred = rf.predict(X_val)
print(f"F1 Score: {f1_score(y_val, y_pred, average='weighted')}")
print(f"Accuracy: {accuracy_score(y_val, y_pred)}")

F1 Score: 1.0
Accuracy: 1.0


In [216]:
svc = SVC(kernel='rbf')
svc.fit(X_train, y_train)
y_pred = svc.predict(X_val)
print(f"F1 Score: {f1_score(y_val, y_pred, average='weighted')}")
print(f"Accuracy: {accuracy_score(y_val, y_pred)}")

F1 Score: 1.0
Accuracy: 1.0


## CARLA

In [208]:
X_carla_disc_fixed, y_carla_disc_fixed = carla_fixed.drop(columns=['label']), np.ones(len(carla_fixed))
X_carla_disc_llm, y_carla_disc_llm = carla_llm.drop(columns=['label']), np.ones(len(carla_llm))

### Fixed

In [211]:
X, y = pd.concat([X_carla_disc_fixed, X_uah_disc_carla], axis=0, ignore_index=True), np.concat([y_carla_disc_fixed, y_uah_disc], axis=0)
X.reset_index(drop=True, inplace=True)
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42, shuffle=True)

In [212]:
rf = RandomForestClassifier()
rf.fit(X_train, y_train)
y_pred = rf.predict(X_val)
print(f"F1 Score: {f1_score(y_val, y_pred, average='weighted')}")
print(f"Accuracy: {accuracy_score(y_val, y_pred)}")

F1 Score: 1.0
Accuracy: 1.0


In [217]:
svc = SVC(kernel='rbf')
svc.fit(X_train, y_train)
y_pred = svc.predict(X_val)
print(f"F1 Score: {f1_score(y_val, y_pred, average='weighted')}")
print(f"Accuracy: {accuracy_score(y_val, y_pred)}")

F1 Score: 1.0
Accuracy: 1.0


### LLM

In [213]:
X, y = pd.concat([X_carla_disc_llm, X_uah_disc_carla], axis=0, ignore_index=True), np.concat([y_carla_disc_llm, y_uah_disc], axis=0)
X.reset_index(drop=True, inplace=True)
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42, shuffle=True)

In [214]:
rf = RandomForestClassifier()
rf.fit(X_train, y_train)
y_pred = rf.predict(X_val)
print(f"F1 Score: {f1_score(y_val, y_pred, average='weighted')}")
print(f"Accuracy: {accuracy_score(y_val, y_pred)}")

F1 Score: 1.0
Accuracy: 1.0


In [218]:
svc = SVC(kernel='rbf')
svc.fit(X_train, y_train)
y_pred = svc.predict(X_val)
print(f"F1 Score: {f1_score(y_val, y_pred, average='weighted')}")
print(f"Accuracy: {accuracy_score(y_val, y_pred)}")

F1 Score: 1.0
Accuracy: 1.0


# Predictive Scores

Here we train on different combinations of real and synthetic data and test on real data only in order to verify the quality of the data for real-world applications.

In [ ]:
PS = {}
n_trials = 8

## Real Only

In [ ]:
columns_to_keep = ['acc_x', 'acc_y', 'acc_z', 'gyro_x', 'gyro_y', 'gyro_z', 'angle', 'speed', 'label']

X_train, y_train = preprocessor.sliding_windows(uah_training[columns_to_keep], window_size=window_size, step_size=step_size)
X_val, y_val = preprocessor.sliding_windows(uah_validation[columns_to_keep], window_size=window_size, step_size=step_size)

scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_val = scaler.transform(X_val)

y_train = preprocessor.one_hot_encode(y_train, one_hot_keys=one_hot_keys)
y_val = preprocessor.one_hot_encode(y_val, one_hot_keys=one_hot_keys)

In [ ]:
# Best model: SVC_{'C': 0.01, 'gamma': 0.01, 'kernel': 'rbf'}
for i in range(n_trials):
    results = create_and_train_model(SVC, {
        'C': 0.01,
        'gamma': 0.01,
        'kernel': 'rbf',
    })
    add_to_dict(PS, 'uah', 'SVC', results)

## SUMO Only

In [12]:
# SUMO
X_sumo_fixed, y_sumo_fixed = preprocessor.sliding_windows(sumo_fixed, window_size=window_size, step_size=step_size)
X_sumo_llm, y_sumo_llm = preprocessor.sliding_windows(sumo_llm, window_size=window_size, step_size=step_size)

In [35]:
# Getting the validation for SUMO variables
print("Columns to keep for validation:", sumo_columns_to_keep)

X_val_base, y_val = preprocessor.sliding_windows(uah_validation[sumo_columns_to_keep], window_size=window_size, step_size=step_size)
y_val = preprocessor.one_hot_encode(y_val, one_hot_keys=one_hot_keys)

Columns to keep for validation: ['acc', 'angle', 'speed', 'gyro_z', 'label']


### Fixed

In [ ]:
X_train, y_train = X_sumo_fixed, y_sumo_fixed

scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_val = scaler.transform(X_val_base)
y_train = preprocessor.one_hot_encode(y_train, one_hot_keys=one_hot_keys)

In [ ]:
# Best model: RandomForestClassifier_{'max_depth': 20, 'n_estimators': 20}
for i in range(n_trials):
    results = create_and_train_model(RandomForestClassifier, {
        'max_depth': 20,
        'n_estimators': 20,
    })
    add_to_dict(PS, 'sumo_fixed', 'RFClassifier', results)

### LLM

In [ ]:
X_train, y_train = X_sumo_llm, y_sumo_llm

scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_val = scaler.transform(X_val_base)
y_train = preprocessor.one_hot_encode(y_train, one_hot_keys=one_hot_keys)

In [ ]:
# Best model: XGBClassifier_{'learning_rate': 0.5, 'max_depth': 5, 'n_estimators': 200}
for i in range(n_trials):
    results = create_and_train_model(xgb.XGBClassifier, {
        'learning_rate': 0.5,
        'max_depth': 5,
        'n_estimators': 200,
    })
    add_to_dict(PS, 'sumo_llm', 'XGBClassifier', results)

## Carla Only

In [32]:
X_carla_fixed, y_carla_fixed = preprocessor.sliding_windows(carla_fixed, window_size=window_size, step_size=step_size)
X_carla_llm, y_carla_llm = preprocessor.sliding_windows(carla_llm, window_size=window_size, step_size=step_size)

In [36]:
# Getting the validation for SUMO variables
print("Columns to keep for validation:", carla_columns_to_keep)
X_val_base, y_val = preprocessor.sliding_windows(uah_validation[carla_columns_to_keep], window_size=window_size, step_size=step_size)
y_val = preprocessor.one_hot_encode(y_val, one_hot_keys=one_hot_keys)

Columns to keep for validation: ['acc_x', 'acc_y', 'acc_z', 'gyro_x', 'gyro_y', 'gyro_z', 'angle', 'label', 'acc']


### Fixed

In [ ]:
X_train, y_train = X_carla_fixed, y_carla_fixed

scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_val = scaler.transform(X_val_base)
y_train = preprocessor.one_hot_encode(y_train, one_hot_keys=one_hot_keys)

In [ ]:
# Best model: RandomForestClassifier_{'max_depth': 10, 'n_estimators': 20}
for i in range(n_trials):
    results = create_and_train_model(RandomForestClassifier, {
        'max_depth': 10,
        'n_estimators': 20,
    })
    add_to_dict(PS, 'carla_fixed', 'RFClassifier', results)

### LLM

In [ ]:
X_train, y_train = X_carla_llm, y_carla_llm

scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_val = scaler.transform(X_val_base)
y_train = preprocessor.one_hot_encode(y_train, one_hot_keys=one_hot_keys)

In [ ]:
# Best model: RandomForestClassifier_{'max_depth': None, 'n_estimators': 500}
for i in range(n_trials):
    results = create_and_train_model(RandomForestClassifier, {
        'max_depth': None,
        'n_estimators': 500,
    })
    add_to_dict(PS, 'carla_llm', 'RFClassifier', results)

## Real + SUMO

In [ ]:
sumo_uah_fixed_20 = preprocessor.fill_synthetic_data(sumo_uah_fixed, 0.2)
sumo_uah_llm_20 = preprocessor.fill_synthetic_data(sumo_uah_llm, 0.2)

sumo_uah_fixed_100 = preprocessor.fill_synthetic_data(sumo_uah_fixed, 1)
sumo_uah_llm_100 = preprocessor.fill_synthetic_data(sumo_uah_llm, 1)

sumo_uah_fixed_200 = preprocessor.fill_synthetic_data(sumo_uah_fixed, 2)
sumo_uah_llm_200 = preprocessor.fill_synthetic_data(sumo_uah_llm, 2)

In [ ]:
X_sumo_uah_fixed_20, y_sumo_uah_fixed_20 = preprocessor.sliding_windows(sumo_uah_fixed_20, window_size=window_size, step_size=step_size)
X_sumo_uah_llm_20, y_sumo_uah_llm_20 = preprocessor.sliding_windows(sumo_uah_llm_20, window_size=window_size, step_size=step_size)

X_sumo_uah_fixed_100, y_sumo_uah_fixed_100 = preprocessor.sliding_windows(sumo_uah_fixed_100, window_size=window_size, step_size=step_size)
X_sumo_uah_llm_100, y_sumo_uah_llm_100 = preprocessor.sliding_windows(sumo_uah_llm_100, window_size=window_size, step_size=step_size)

X_sumo_uah_fixed_200, y_sumo_uah_fixed_200 = preprocessor.sliding_windows(sumo_uah_fixed_200, window_size=window_size, step_size=step_size)
X_sumo_uah_llm_200, y_sumo_uah_llm_200 = preprocessor.sliding_windows(sumo_uah_llm_200, window_size=window_size, step_size=step_size)

In [ ]:
print("Columns to keep for validation:", sumo_columns_to_keep)

X_val_base, y_val = preprocessor.sliding_windows(uah_validation[sumo_columns_to_keep], window_size=window_size, step_size=step_size)
y_val = preprocessor.one_hot_encode(y_val, one_hot_keys=one_hot_keys)

Columns to keep for validation: ['acc', 'angle', 'speed', 'gyro_z', 'label']


## Real + SUMO (20%)

### Fixed

In [ ]:
X_train, y_train = X_sumo_uah_fixed_20, y_sumo_uah_fixed_20

scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_val = scaler.transform(X_val_base)
y_train = preprocessor.one_hot_encode(y_train, one_hot_keys=one_hot_keys)

In [ ]:
# Best model: SVC_{'C': 0.1, 'gamma': 0.01, 'kernel': 'rbf'}
for i in range(n_trials):
    results = create_and_train_model(SVC, {
        'C': 0.1,
        'gamma': 0.01,
        'kernel': 'rbf',
    })
    add_to_dict(PS, 'sumo_uah_fixed_20', 'SVC', results)

### LLM

In [ ]:
X_train, y_train = X_sumo_uah_llm_20, y_sumo_uah_llm_20

scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_val = scaler.transform(X_val_base)
y_train = preprocessor.one_hot_encode(y_train, one_hot_keys=one_hot_keys)

In [ ]:
# Best model: SVC_{'C': 0.1, 'gamma': 0.1, 'kernel': 'rbf'}
for i in range(n_trials):
    results = create_and_train_model(SVC, {
        'C': 0.1,
        'gamma': 0.1,
        'kernel': 'rbf',
    })
    add_to_dict(PS, 'sumo_uah_llm_20', 'SVC', results)

## Real + SUMO (100%)

### Fixed

In [ ]:
X_train, y_train = X_sumo_uah_fixed_100, y_sumo_uah_fixed_100

scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_val = scaler.transform(X_val_base)
y_train = preprocessor.one_hot_encode(y_train, one_hot_keys=one_hot_keys)

In [ ]:
# Best model: SVC_{'C': 1, 'gamma': 0.001, 'kernel': 'rbf'}
for i in range(n_trials):
    results = create_and_train_model(SVC, {
        'C': 1,
        'gamma': 0.001,
        'kernel': 'rbf',
    })
    add_to_dict(PS, 'sumo_uah_fixed_100', 'SVC', results)

### LLM

In [ ]:
X_train, y_train = X_sumo_uah_llm_100, y_sumo_uah_llm_100

scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_val = scaler.transform(X_val_base)
y_train = preprocessor.one_hot_encode(y_train, one_hot_keys=one_hot_keys)

In [ ]:
# Best model: SVC_{'C': 0.01, 'gamma': 0.1, 'kernel': 'rbf'}
for i in range(n_trials):
    results = create_and_train_model(SVC, {
        'C': 0.01,
        'gamma': 0.1,
        'kernel': 'rbf',
    })
    add_to_dict(PS, 'sumo_uah_llm_100', 'SVC', results)

## Real + SUMO (200%)

### Fixed

In [ ]:
X_train, y_train = X_sumo_uah_fixed_200, y_sumo_uah_fixed_200

scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_val = scaler.transform(X_val_base)
y_train = preprocessor.one_hot_encode(y_train, one_hot_keys=one_hot_keys)

# TODO: TRAIN MODEL

In [ ]:
# Best model: SVC_{'C': 0.01, 'gamma': 0.01, 'kernel': 'rbf'}
for i in range(n_trials):
    results = create_and_train_model(SVC, {
        'C': 0.01,
        'gamma': 0.01,
        'kernel': 'rbf',
    })
    add_to_dict(PS, 'sumo_uah_fixed_200', 'SVC', results)

### LLM

In [ ]:
X_train, y_train = X_sumo_uah_llm_200, y_sumo_uah_llm_200

scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_val = scaler.transform(X_val_base)
y_train = preprocessor.one_hot_encode(y_train, one_hot_keys=one_hot_keys)

In [ ]:
# Best model: SVC_{'C': 0.5, 'gamma': 0.001, 'kernel': 'rbf'}
for i in range(n_trials):
    results = create_and_train_model(SVC, {
        'C': 0.5,
        'gamma': 0.001,
        'kernel': 'rbf',
    })
    add_to_dict(PS, 'sumo_uah_llm_200', 'SVC', results)

## Real + CARLA

In [ ]:
carla_uah_fixed_20 = preprocessor.fill_synthetic_data(carla_uah_fixed, 0.2)
carla_uah_llm_20 = preprocessor.fill_synthetic_data(carla_uah_llm, 0.2)

carla_uah_fixed_100 = preprocessor.fill_synthetic_data(carla_uah_fixed, 0.5)
carla_uah_llm_100 = preprocessor.fill_synthetic_data(carla_uah_llm, 0.5)

carla_uah_fixed_200 = preprocessor.fill_synthetic_data(carla_uah_fixed, 0.7)
carla_uah_llm_200 = preprocessor.fill_synthetic_data(carla_uah_llm, 0.7)

In [ ]:
X_carla_uah_fixed_20, y_carla_uah_fixed_20 = preprocessor.sliding_windows(carla_uah_fixed_20, window_size=window_size, step_size=step_size)
X_carla_uah_llm_20, y_carla_uah_llm_20 = preprocessor.sliding_windows(carla_uah_llm_20, window_size=window_size, step_size=step_size)

X_carla_uah_fixed_100, y_carla_uah_fixed_100 = preprocessor.sliding_windows(carla_uah_fixed_100, window_size=window_size, step_size=step_size)
X_carla_uah_llm_100, y_carla_uah_llm_100 = preprocessor.sliding_windows(carla_uah_llm_100, window_size=window_size, step_size=step_size)

X_carla_uah_fixed_200, y_carla_uah_fixed_200 = preprocessor.sliding_windows(carla_uah_fixed_200, window_size=window_size, step_size=step_size)
X_carla_uah_llm_200, y_carla_uah_llm_200 = preprocessor.sliding_windows(carla_uah_llm_200, window_size=window_size, step_size=step_size)

In [ ]:
print("Columns to keep for validation:", carla_columns_to_keep)

X_val_base, y_val = preprocessor.sliding_windows(uah_validation[carla_columns_to_keep], window_size=window_size, step_size=step_size)
y_val = preprocessor.one_hot_encode(y_val, one_hot_keys=one_hot_keys)

Columns to keep for validation: ['acc_x', 'acc_y', 'acc_z', 'gyro_x', 'gyro_y', 'gyro_z', 'angle', 'label']


## Real + CARLA (20%)

### Fixed

In [ ]:
X_train, y_train = X_carla_uah_fixed_20, y_carla_uah_fixed_20

scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_val = scaler.transform(X_val_base)
y_train = preprocessor.one_hot_encode(y_train, one_hot_keys=one_hot_keys)

In [ ]:
# Best model: RandomForestClassifier_{'max_depth': 20, 'n_estimators': 20}
for i in range(n_trials):
    results = create_and_train_model(RandomForestClassifier, {
        'max_depth': 20,
        'n_estimators': 20,
    })
    add_to_dict(PS, 'carla_uah_fixed_20', 'RFClassifier', results)

### LLM

In [ ]:
X_train, y_train = X_carla_uah_llm_20, y_carla_uah_llm_20

scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_val = scaler.transform(X_val_base)
y_train = preprocessor.one_hot_encode(y_train, one_hot_keys=one_hot_keys)

# TODO: TRAIN MODEL

In [ ]:
# Best model: RandomForestClassifier_{'max_depth': 20, 'n_estimators': 50}
for i in range(n_trials):
    results = create_and_train_model(RandomForestClassifier, {
        'max_depth': 20,
        'n_estimators': 50,
    })
    add_to_dict(PS, 'carla_uah_llm_20', 'RFClassifier', results)

## Real + CARLA (100%)

### Fixed

In [ ]:
X_train, y_train = X_carla_uah_fixed_100, y_carla_uah_fixed_100

scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_val = scaler.transform(X_val_base)
y_train = preprocessor.one_hot_encode(y_train, one_hot_keys=one_hot_keys)

In [ ]:
# Best model: XGBClassifier_{'colsample_bytree': 0.7, 'learning_rate': 0.1, 'max_depth': 5, 'n_estimators': 200, 'subsample': 0.7}
for i in range(n_trials):
    results = create_and_train_model(xgb.XGBClassifier, {
        'colsample_bytree': 0.7,
        'learning_rate': 0.1,
        'max_depth': 5,
        'n_estimators': 200,
        'subsample': 0.7,
    })
    add_to_dict(PS, 'carla_uah_fixed_100', 'XGBClassifier', results)

### LLM

In [ ]:
X_train, y_train = X_carla_uah_llm_100, y_carla_uah_llm_100

scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_val = scaler.transform(X_val_base)
y_train = preprocessor.one_hot_encode(y_train, one_hot_keys=one_hot_keys)

# TODO: TRAIN MODEL

In [ ]:
# Best model: XGBClassifier_{'colsample_bytree': 1.0, 'learning_rate': 0.3, 'max_depth': 5, 'n_estimators': 500, 'subsample': 0.7}
for i in range(n_trials):
    results = create_and_train_model(xgb.XGBClassifier, {
        'colsample_bytree': 1.0,
        'learning_rate': 0.3,
        'max_depth': 5,
        'n_estimators': 500,
        'subsample': 0.7,
    })
    add_to_dict(PS, 'carla_uah_llm_100', 'XGBClassifier', results)

## Real + CARLA (200%)

### Fixed

In [ ]:
X_train, y_train = X_carla_uah_fixed_200, y_carla_uah_fixed_200

scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_val = scaler.transform(X_val_base)
y_train = preprocessor.one_hot_encode(y_train, one_hot_keys=one_hot_keys)

In [ ]:
# Best model: XGBClassifier_{'colsample_bytree': 1.0, 'learning_rate': 0.1, 'max_depth': 5, 'n_estimators': 200, 'subsample': 1.0}
for i in range(n_trials):
    results = create_and_train_model(xgb.XGBClassifier, {
        'colsample_bytree': 1.0,
        'learning_rate': 0.1,
        'max_depth': 5,
        'n_estimators': 200,
        'subsample': 1.0,
    })
    add_to_dict(PS, 'carla_uah_fixed_200', 'XGBClassifier', results)

### LLM

In [ ]:
X_train, y_train = X_carla_uah_llm_200, y_carla_uah_llm_200

scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_val = scaler.transform(X_val_base)
y_train = preprocessor.one_hot_encode(y_train, one_hot_keys=one_hot_keys)

In [ ]:
# Best model: RandomForestClassifier_{'max_depth': 50, 'n_estimators': 20}
for i in range(n_trials):
    results = create_and_train_model(RandomForestClassifier, {
        'max_depth': 50,
        'n_estimators': 20,
    })
    add_to_dict(PS, 'carla_uah_llm_200', 'RandomForestClassifier', results)